# Análise de Dados com Python - Word Clouds

**Minicurso:** Análise de Dados com Python

**Instrutor:** Humberto da Silva Neto

**Aluno:** 

---

In [0]:
import matplotlib.pyplot as plt

**Word clouds** (também conhecidas como nuvens de texto ou nuvens de tags) funcionam de uma forma simples: quanto mais uma palavra específica aparece em uma fonte de dados textuais (como um discurso, um blog ou um banco de dados), maior e mais arrojada aparece na nuvem de palavras.


Felizmente, já existe uma biblioteca em Python para gerar essa *nuvens de palavra*. A biblioteca, chamada **`word_cloud`** foi desenvolvida por Andreas Mueller. Você pode aprender mais sobre a biblioteca seguindo este [link](https://github.com/amueller/word_cloud/).

Vamos usar essa biblioteca para aprender como gerar uma nuvem de palavras para um determinado documento de texto.

Para isso, antes installe a biblioteca com o comando:

```sh
!pip install wordcloud
```

In [0]:
# instale a biblioteca wordcloud
!pip install wordcloud

Agora import a biblioteca com o comando abaixo:

```python
from wordcloud import WordCloud, STOPWORDS
```

In [0]:
# importe a biblioteca
from wordcloud import WordCloud, STOPWORDS

print('Wordcloud foi instalado e importado!')

Wordcloud são comumente usadas para executar análises de alto nível e visualização de dados de texto. De acordo, vamos fazer uma digressão do conjunto de dados de imigração e trabalhar com um exemplo que envolve a análise de dados de texto. 

Vamos tentar analisaro sétimo livro da série Harry Potter, escrito pela autora britânica J. K. Rowling, "**Harry Potter and the Deathly Hallows** (em português, Harry Potter e as Relíquias da Morte)"

Para isso, iremos precisaremos baixar o arquivo .txt do livro.

In [0]:
import codecs

# download do arquivo
!wget https://raw.githubusercontent.com/hsneto/py-pandas-minicourse/master/datasets/Harry%20Potter%20and%20the%20Deathly%20Hallows.txt -O harry_potter.txt
  
# abra o arquivo e leia-o em uma variável hp
hp = codecs.open('harry_potter.txt', 'r', encoding='utf-8', errors='ignore').read()

print('Arquivo baixado e carregado')

Crie um objeto **wordcloud** e gere uma nuvem de palavras. Por simplicidade, vamos gerar uma nuvem de palavras usando apenas as primeiras 5000 palavras do romance.

In [0]:
# instanciar um objeto de nuvem de palavras
hp_wc = WordCloud(
    background_color='white',
    max_words=5000,
    stopwords=STOPWORDS
)

# gerar a nuvem de palavras
hp_wc.generate(hp)

In [0]:
# exibir a nuvem de palavras

plt.imshow(hp_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

É possível notar que as palavras mais comuns (entre as 5000 primeiras) são Harry, Ron e Hermione. Agora, vamos supor que algumas das palavras destacadas não são  muito informativas ou do meu interesse, é possível associar essa palavra ao conjunto **STOPWORDS**$^{[1]}$ para que essa seja ignorada.

Por exemplo, vamos ignorar uma palavra de destaque como **now**. Para isso usaremos o comando:

```python
STOPWORDS.add('now')`
```

e iremos gerar a nuvem de palavras de novo. 

---
$^{[1]}$ Para conferir as palavras  padrões do conjunto **STOPWORDS** clique [aqui](https://github.com/amueller/word_cloud/blob/master/wordcloud/stopwords).

In [0]:
STOPWORDS.add('now')

# instanciar um objeto de nuvem de palavras
hp_wc = WordCloud(
    background_color='white',
    max_words=5000,
    stopwords=STOPWORDS
)

# gerar a nuvem de palavras
hp_wc.generate(hp)

# exibir a nuvem de palavras

plt.imshow(hp_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Como esperado, não encontramos a palavra **now** na nuvem agora.

Excelente! Outra coisa legal que você pode implementar com a biblioteca **`word_cloud`** é sobrepor as palavras em uma máscara de qualquer forma.

Já que estamos usando um livro de Harry Potter, nada mais justo que usar máscaras relacionadas ao tema. Para isso eu preparei uma função chamada de **`get_bw_img`**. Essa função  para preparar essa máscara em função de uma imagem online. 

A função recebe como argumento o url de uma imagem e retorna uma máscara (uma imagem binarizada, ou seja, em preto e branco) da mesma.

---
*Obs.: Para o nosso objetivo, tenha em mente em que as palavras da nuvem estarão dispostas na parte preta da máscara.*



In [0]:
import urllib.request
from numpy import asarray, ones, uint8
from cv2 import imdecode, dilate, threshold, THRESH_BINARY, THRESH_OTSU

def get_bw_img(url, kernel_size=None, inv=False, show=False):
  '''
  Return a black and white image from a url link.
  
  Args:
    url (str)           --> link to the image.
    kernel_size (tupla) --> kernel size to dilate the image.
    inv (bool)          --> invert black and white colors
    show (bool)         --> plot both, the input (greyscale) and output image
  '''
  
  def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urllib.request.urlopen(url)
    image = asarray(bytearray(resp.read()), dtype="uint8")
    image = imdecode(image, 0) # 0 to load grayscale image

    # return the image
    return image
  
  def img2bw(img, inv=True):
    # apply Otsu's thresholding
    _,th2 = threshold(img,0,255,THRESH_BINARY + THRESH_OTSU)
    return th2
  
  img = url_to_image(url)
  bw_img = img2bw(img, inv)
  
  if kernel_size is not None:
    kernel = ones(kernel_size, uint8)
    bw_img = dilate(bw_img, kernel,iterations = 1)
    
  if inv:
    bw_img = 255-bw_img
  
  if show:
    # plot the output image
    plt.subplot(121)
    plt.imshow(img, cmap=plt.cm.gray, interpolation='bilinear')
    plt.subplot(122)
    plt.imshow(bw_img, cmap=plt.cm.gray, interpolation='bilinear')
    plt.show()

  return bw_img

In [0]:
mask1 = get_bw_img(url='http://cdn.shopify.com/s/files/1/2597/5112/products/hptvthogjt_1_1024x1024.jpg?v=1517442756', 
                 kernel_size=(25,25), 
                 inv=True, 
                 show=True)

mask2 = get_bw_img(url='https://images.ctfassets.net/bxd3o8b291gf/3SQ3X2km8wkQIsQWa02yOY/25f258f21bdbe5f552a4419bb775f4f0/HarryPotter_WB_F4_HarryPotterMidshot_Promo_080615_Port.jpg?w=1200', 
                 kernel_size=None, 
                 inv=False, 
                 show=True)

Agora iremos visualizar a nuvem em cima das máscaras criadas acima. Caso você tenha interesse em controlar a palheta de cores das palavras plotadas, sinta-se livre para alterar o argumento **colormap**. 

Os links abaixo contém exemplos das palhetas de cores disponíveis:

- [link1](https://matplotlib.org/examples/color/colormaps_reference.html)
- [link2](https://matplotlib.org/users/colormaps.html)

In [0]:
# instanciar um objeto de nuvem de palavras
hp_wc_1 = WordCloud(background_color='black', max_words=5000, 
                    mask=mask1, stopwords=STOPWORDS, colormap=plt.cm.BuPu)

hp_wc_2 = WordCloud(mode='RGBA', background_color=None, max_words=5000, 
                    mask=mask2, stopwords=STOPWORDS, colormap=plt.cm.flag)

# gerar a nuvem de palavras
hp_wc_1.generate(hp)
hp_wc_2.generate(hp)

# exibir a nuvem de palavras
plt.subplot(121)
plt.imshow(hp_wc_1, interpolation='bilinear')
plt.axis('off')

plt.subplot(122)
plt.imshow(hp_wc_2, interpolation='bilinear')
plt.axis('off')

plt.show()